In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import sys
sys.path.append("../")
from PythonCode.preprocess.preprocess import *
from PythonCode.preprocess.complexStyleFeatures import *
from PythonCode.preprocess.simpleStyleFeatures import *
import seaborn as sns
import gensim
from gensim.utils import simple_preprocess
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pickle 
import gensim.downloader
from pprint import pprint
DATA_PATH_TRAIN = "../Data/C50/C50train/"
DATA_PATH_TEST = "../Data/C50/C50test/"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yosef\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import pickle
import tensorflow as tf
import numpy as np
import nltk
import pandas as pd
import gensim.downloader
import gensim
import re
from typing import Optional
import swifter
from PythonCode.preprocess.preprocess import load_data
from sklearn.model_selection import train_test_split
from keras.layers import Dense, GRU, AvgPool1D
from keras.models import Sequential
from sklearn.metrics import classification_report
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
print("downloading pretrained embedding model.\nthis may take a while...")
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

downloading pretrained embedding model.
this may take a while...


In [ ]:
EMBEDDING_SIZE = 50
NUM_OF_SENTENCE_CHUNK = 3
MAX_LENGTH = 170
TEST_PART = 0.1
VALIDATION_PART = 0.2
MAX_SENTENCE_LENGTH = 100
MAX_NUMBER_OF_SENTENCE = 100
DATA_PATH = "../Data/C50"

In [9]:
from keras.layers import Masking

def tranform_word(word: str) -> Optional[np.ndarray]:
    word = re.sub(r'[^a-z]', '', word.lower())
    if word in glove_vectors:
        return glove_vectors[word]
    return None


def complex_tranform_word(word: str):
    result = tranform_word(word)
    if result is None:
        token = lemmatizer.lemmatize(word)
        if token in glove_vectors:
            return glove_vectors[token]
        token = stemmer.stem(word)
        if token in glove_vectors:
            return glove_vectors[token]
    return result


def pad_matrix(arr: np.ndarray, max_length: int) -> Optional[np.ndarray]:
    if arr.size == 0:
        return None
    if arr.shape[0] == max_length:
        return arr
    if arr.shape[0] > max_length:
        return arr[:max_length, :]
    return np.concatenate([arr, np.zeros((max_length - arr.shape[0], arr.shape[1]))], axis=0, dtype=float)


def get_datasets(data_path: str = "../Data/C50") -> pd.DataFrame:
    df_test = load_data(f"{data_path}/C50test", 50)
    df_train = load_data(f"{data_path}/C50train", 50)
    return df_train.append(df_test, ignore_index=True)


def preprocess_labels(y: pd.Series) -> np.ndarray:
    y_codes = pd.Categorical(y).codes
    one_hot = tf.keras.utils.to_categorical(
        y_codes, num_classes=pd.Series(y_codes).unique().size, dtype='float32'
    )
    return np.expand_dims(one_hot, axis=1)


def pad_array(arr: np.ndarray, pad_size: int):  # TODO: reuse pad_matrix instead
    if arr.size == pad_size:
        return arr
    elif arr.size > pad_size:
        return arr[:pad_size, ]
    return np.concatenate([arr, np.zeros(pad_size - arr.size)], dtype=float)


def article_level_preprocess_helper(text: str):
    sentences = nltk.sent_tokenize(text)
    result = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        curr_result = []
        for word in words:
            embedding = tranform_word(word)
            if embedding is not None:
                curr_result.append(embedding)
        if len(curr_result) != 0:
            result.append(pad_array(np.array(curr_result, dtype=float).mean(axis=1, dtype=float), MAX_SENTENCE_LENGTH))
    return pad_matrix(np.array(result), MAX_NUMBER_OF_SENTENCE)


def article_level_preprocess(df: pd.DataFrame):
    def helper(X):
        res = X.swifter.apply(article_level_preprocess_helper).reset_index(drop=True)
        return np.vstack(res).reshape((res.size, MAX_NUMBER_OF_SENTENCE, MAX_SENTENCE_LENGTH))

    X_train, X_test, y_train, y_test = train_test_split(df["book_text"], df["author_name"], test_size=TEST_PART)
    return helper(X_train), helper(X_test), preprocess_labels(y_train), preprocess_labels(y_test)


In [10]:
df = get_datasets(DATA_PATH)
X_train, X_test, y_train, y_test = article_level_preprocess(df)
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(MAX_NUMBER_OF_SENTENCE, MAX_SENTENCE_LENGTH)))
model.add(GRU(100, recurrent_dropout=0.2,return_sequences=True))
model.add(AvgPool1D(pool_size=(MAX_NUMBER_OF_SENTENCE,)))
model.add(Dense(50, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
print(model.summary())

Pandas Apply:   0%|          | 0/4500 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/500 [00:00<?, ?it/s]

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 100, 100)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 100, 100)          60600     
_________________________________________________________________
average_pooling1d_1 (Average (None, 1, 100)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 50)             5050      
Total params: 65,650
Trainable params: 65,650
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VALIDATION_PART)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
model_name = "article_based_model"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f"./{model_name}-checkpoints",
                                                               save_weights_only=False,
                                                               monitor='val_accuracy', mode='max',
                                                               save_best_only=True)
history = model.fit(x=X_train, y=y_train, epochs=30000, shuffle=True,
                    batch_size=200, validation_data=(X_val, y_val), callbacks=[callback, model_checkpoint_callback])
with open(f"{model_name}-history", "w") as file:
    pickle.dump(history, file)
model.save(model_name)
y_pred = model.predict(X_test)
print(classification_report(y_pred, y_test))

Epoch 1/30000
18/18 [==============================] - 16s 800ms/step - loss: 3.9101 - accuracy: 0.0303 - val_loss: 3.9073 - val_accuracy: 0.0411
INFO:tensorflow:Assets written to: .\article_based_model-checkpoints\assets
Epoch 2/30000
18/18 [==============================] - 12s 686ms/step - loss: 3.8943 - accuracy: 0.0564 - val_loss: 3.8691 - val_accuracy: 0.0311
Epoch 3/30000
18/18 [==============================] - 13s 740ms/step - loss: 3.7800 - accuracy: 0.0425 - val_loss: 3.7461 - val_accuracy: 0.0344
Epoch 4/30000
18/18 [==============================] - 16s 912ms/step - loss: 3.7307 - accuracy: 0.0444 - val_loss: 3.7248 - val_accuracy: 0.0378
Epoch 5/30000
18/18 [==============================] - 18s 1s/step - loss: 3.7115 - accuracy: 0.0489 - val_loss: 3.7065 - val_accuracy: 0.0378
Epoch 6/30000
18/18 [==============================] - 16s 869ms/step - loss: 3.6953 - accuracy: 0.0589 - val_loss: 3.6966 - val_accuracy: 0.0400
Epoch 7/30000
18/18 [==============================

Epoch 50/30000
18/18 [==============================] - 19s 1s/step - loss: 3.2548 - accuracy: 0.1586 - val_loss: 3.4311 - val_accuracy: 0.1089
Epoch 51/30000
18/18 [==============================] - 19s 1s/step - loss: 3.2476 - accuracy: 0.1542 - val_loss: 3.4195 - val_accuracy: 0.1100
Epoch 52/30000
18/18 [==============================] - 19s 1s/step - loss: 3.2423 - accuracy: 0.1558 - val_loss: 3.4123 - val_accuracy: 0.1089
Epoch 53/30000
18/18 [==============================] - 19s 1s/step - loss: 3.2301 - accuracy: 0.1614 - val_loss: 3.4178 - val_accuracy: 0.1178
INFO:tensorflow:Assets written to: .\article_based_model-checkpoints\assets
Epoch 54/30000
18/18 [==============================] - 19s 1s/step - loss: 3.2255 - accuracy: 0.1614 - val_loss: 3.4291 - val_accuracy: 0.1167
Epoch 55/30000
18/18 [==============================] - 18s 978ms/step - loss: 3.2223 - accuracy: 0.1536 - val_loss: 3.4091 - val_accuracy: 0.1089
Epoch 56/30000
18/18 [==============================] - 1

TypeError: write() argument must be str, not bytes